<h1 style="color:red"> Création et peuplement d'une base de données à partir d'un fichier CSV </h1>

<h2><b> 1) Import des librairies </b></h2>

In [3]:
import pandas as pd
import numpy as np
import psycopg2

<h2><b> 2) Chargement du fichier CSV </b></h2>

In [4]:
df = pd.read_csv("consoelecgaz2024.csv", sep="\t")

df.head()

,OPERATEUR,FILIERE,Année,Code Département,Nom Département,Code Région,Nom Région,CODE CATEGORIE CONSOMMATION,CODE GRAND SECTEUR,Nb sites,Conso totale (MWh),Conso moyenne (MWh)
0,RTE,Electricité,2016,61,Orne,28,Normandie,ENT,INDUSTRIE,4.0,"101127,32","25281,83"
1,RTE,Electricité,2016,62,Pas-de-Calais,32,Hauts-de-France,ENT,TERTIAIRE,9.0,"701256,73","77917,4144444444"
2,RTE,Electricité,2016,64,Pyrénées-Atlantiques,75,Nouvelle-Aquitaine,ENT,TERTIAIRE,8.0,"25220,57","3152,57125"
3,RTE,Electricité,2016,66,Pyrénées-Orientales,76,Occitanie,ENT,TERTIAIRE,15.0,"26684,75","1778,98333333333"
4,RTE,Electricité,2016,68,Haut-Rhin,44,Grand Est,ENT,TERTIAIRE,2.0,"65937,45","32968,725"


<h2><b> 3) Prise d'information des données du fichier </b></h2>

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104086 entries, 0 to 104085
Data columns (total 12 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   OPERATEUR                    104086 non-null  object 
 1   FILIERE                      104086 non-null  object 
 2   Année                        104086 non-null  int64  
 3   Code Département             104086 non-null  object 
 4   Nom Département              104086 non-null  object 
 5   Code Région                  104086 non-null  int64  
 6   Nom Région                   104086 non-null  object 
 7   CODE CATEGORIE CONSOMMATION  104086 non-null  object 
 8   CODE GRAND SECTEUR           104086 non-null  object 
 9   Nb sites                     103005 non-null  float64
 10  Conso totale (MWh)           103005 non-null  object 
 11  Conso moyenne (MWh)          103005 non-null  object 
dtypes: float64(1), int64(2), object(9)
memory usage: 9.5+ MB


<h2><b> 4) Modification des types de données </b></h2>

<h3 style="color:blue"><b> 4.1) Modification pour la consommation totale et moyenne en (MWh)</b> </h3>

In [7]:
df["Conso totale (MWh)"] = df["Conso totale (MWh)"].astype(str).str.replace(",", ".").astype(float)
df["Conso moyenne (MWh)"] = df["Conso moyenne (MWh)"].astype(str).str.replace(",", ".").astype(float)

In [8]:
df[["Conso totale (MWh)","Conso moyenne (MWh)"]].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104086 entries, 0 to 104085
Data columns (total 2 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   Conso totale (MWh)   103005 non-null  float64
 1   Conso moyenne (MWh)  103005 non-null  float64
dtypes: float64(2)
memory usage: 1.6 MB


<h3 style="color:blue"><b> 4.2) Modification pour les codes "Département" et "Région"</b> </h3>

In [9]:
df["Code Département"] = df["Code Département"].astype(str)
df["Code Région"] = df["Code Région"].astype(str)

In [10]:
df[["Code Département","Code Région"]].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104086 entries, 0 to 104085
Data columns (total 2 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   Code Département  104086 non-null  object
 1   Code Région       104086 non-null  object
dtypes: object(2)
memory usage: 1.6+ MB


<h2><b> 5) Prise d'information des NaN </b></h2>

In [11]:
df.isna().sum()

OPERATEUR                         0
FILIERE                           0
Année                             0
Code Département                  0
Nom Département                   0
Code Région                       0
Nom Région                        0
CODE CATEGORIE CONSOMMATION       0
CODE GRAND SECTEUR                0
Nb sites                       1081
Conso totale (MWh)             1081
Conso moyenne (MWh)            1081
dtype: int64

<h2><b> 6) Remplissage des valeurs manquantes </b></h2>

In [12]:
df["Nb sites"] = df["Nb sites"].fillna(df["Nb sites"].median())
df["Conso totale (MWh)"] = df["Conso totale (MWh)"].fillna(df["Conso totale (MWh)"].median())
df["Conso moyenne (MWh)"] = df["Conso moyenne (MWh)"].fillna(df["Conso moyenne (MWh)"].median())

<h2><b> 7) Connection à la base de données </b></h2>

In [13]:
try:

    conn = psycopg2.connect(
        dbname="consommation_elec",
        user="postgres",
        password="0000",
        host="localhost",
        port="5432"
    )

except psycopg2.OperationalError as error:
    print(f"OperationalError: {error}")

<h2><b> 8) Supression des doublons pour le remplissage des tables de Dimensions </b></h2>

In [14]:
def format_float(value):
    if isinstance(value, str):
        return float(value.replace(",", ".")) 
    return value  

unique_rows_filiere = df.drop_duplicates(subset=['FILIERE'])
unique_rows_temps = df.drop_duplicates(subset=['Année'])
unique_rows_secteur = df.drop_duplicates(subset=['CODE GRAND SECTEUR'])
unique_rows_region = df.drop_duplicates(subset=["Code Région","Nom Région"])
unique_rows_departement = df.drop_duplicates(subset=["Code Département","Nom Département"])
unique_rows_consommation = df.drop_duplicates(subset=["Nb sites", "Conso totale (MWh)", "Conso moyenne (MWh)","OPERATEUR","CODE CATEGORIE CONSOMMATION"])

In [15]:
cursor = conn.cursor()

<h2><b> 9) Création de la base de données </b></h2>

In [ ]:
cursor.execute(
    """
-- Dimension : REGION
CREATE TABLE region (
    region_id INTEGER GENERATED BY DEFAULT AS IDENTITY PRIMARY KEY,
    code_region TEXT NOT NULL UNIQUE,
    nom_region TEXT NOT NULL UNIQUE
);

-- Dimension : DEPARTEMENT
CREATE TABLE departement (
    departement_id INTEGER GENERATED BY DEFAULT AS IDENTITY PRIMARY KEY,
    region_id INTEGER NOT NULL REFERENCES region(region_id) ON DELETE CASCADE,
    code_departement TEXT NOT NULL,
    nom_departement TEXT NOT NULL,
    CONSTRAINT unique_code_nom_departement UNIQUE (code_departement, nom_departement)
);

-- Dimension : SECTEUR
CREATE TABLE secteur (
    secteur_id INTEGER GENERATED BY DEFAULT AS IDENTITY PRIMARY KEY,
    type_secteur TEXT NOT NULL UNIQUE
);

-- Dimension : TEMPS
CREATE TABLE temps (
    temps_id INTEGER GENERATED BY DEFAULT AS IDENTITY PRIMARY KEY,
    annee INTEGER NOT NULL UNIQUE
);

-- Dimension : FILLIERE
CREATE TABLE filiere (
    filiere_id INTEGER GENERATED BY DEFAULT AS IDENTITY PRIMARY KEY,
    type_filiere TEXT NOT NULL UNIQUE
);

-- Table de faits : CONSOMMATION
CREATE TABLE consommation (
    consommation_id INTEGER GENERATED BY DEFAULT AS IDENTITY PRIMARY KEY,
    region_id INTEGER NOT NULL REFERENCES region(region_id) ON DELETE CASCADE,
    departement_id INTEGER NOT NULL REFERENCES departement(departement_id) ON DELETE CASCADE,
    temps_id INTEGER NOT NULL REFERENCES temps(temps_id) ON DELETE CASCADE,
    filiere_id INTEGER NOT NULL REFERENCES filiere(filiere_id) ON DELETE CASCADE,
    secteur_id INTEGER NOT NULL REFERENCES secteur(secteur_id),
    categorie_consommation TEXT NOT NULL,
    operateur TEXT NOT NULL,
    nb_site INTEGER,
    conso_totale FLOAT,
    conso_moyenne FLOAT
);
"""
)

<h2><b> 10) Peuplement de la base de données </b></h2>

In [16]:

# Vérifier si les tables sont vides avant insertion
tables_to_check = {
    "filiere": unique_rows_filiere,
    "temps": unique_rows_temps,
    "secteur": unique_rows_secteur,
    "region": unique_rows_region,
    "departement": unique_rows_departement,
    "consommation" : unique_rows_consommation
}

for table_name, data in tables_to_check.items():
    cursor.execute(f"SELECT COUNT(*) FROM {table_name};")
    count = cursor.fetchone()[0]  # Récupérer le nombre de lignes

    if count == 0:  # Seulement si la table est vide
        for _, row in data.iterrows():
            if table_name == "filiere":
                cursor.execute("""
                    INSERT INTO filiere (type_filiere) VALUES (%s);
                """, (row['FILIERE'],))
            elif table_name == "temps":
                cursor.execute("""
                    INSERT INTO temps (annee) VALUES (%s);
                """, (row["Année"],))
            elif table_name == "secteur":
                cursor.execute("""
                    INSERT INTO secteur (type_secteur) VALUES (%s);
                """, (row["CODE GRAND SECTEUR"],))
            elif table_name == "region":
                cursor.execute(""" 
                    INSERT INTO region (code_region, nom_region) VALUES (%s,%s);
                """, (row["Code Région"], row["Nom Région"]))
            elif table_name == "departement":
                cursor.execute("""
                    SELECT region_id FROM region WHERE code_region = %s;
                """, (row["Code Région"],))  # Recherche de l'ID de la région

                region_id = cursor.fetchone()  # Récupération de l'ID

                if region_id:  # Vérifie que la région existe avant d'insérer
                    cursor.execute("""
                        INSERT INTO departement (code_departement, nom_departement, region_id)
                        VALUES (%s, %s, %s);
                    """, (row["Code Département"], row["Nom Département"], region_id[0]))
            else:
                cursor.execute("SELECT region_id FROM region WHERE code_region = %s;", (row["Code Région"],))
                region_id = cursor.fetchone()
                
                cursor.execute("SELECT departement_id FROM departement WHERE code_departement = %s;", (row["Code Département"],))
                departement_id = cursor.fetchone()
                
                cursor.execute("SELECT temps_id FROM temps WHERE annee = %s;", (row["Année"],))
                temps_id = cursor.fetchone()

                cursor.execute("SELECT filiere_id FROM filiere WHERE type_filiere = %s;", (row["FILIERE"],))
                filiere_id = cursor.fetchone()
                
                cursor.execute("SELECT secteur_id FROM secteur WHERE type_secteur = %s;", (row["CODE GRAND SECTEUR"],))
                secteur_id = cursor.fetchone()

                # Vérifier que toutes les références existent avant d'insérer
                if region_id and departement_id and temps_id and secteur_id and filiere_id:
                    cursor.execute("""
                        INSERT INTO consommation (
                            region_id, departement_id, temps_id, filiere_id, secteur_id,
                            categorie_consommation, operateur, nb_site, conso_totale, conso_moyenne
                        ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s,%s);
                    """, (
                        region_id[0], departement_id[0], temps_id[0], filiere_id[0], secteur_id[0], 
                        row["CODE CATEGORIE CONSOMMATION"], row["OPERATEUR"], format_float(row["Nb sites"]),
                        format_float(row["Conso totale (MWh)"]),
                        format_float(row["Conso moyenne (MWh)"])
                    ))


conn.commit()
cursor.close()
conn.close()
